---
## Setup and Configuration

# Content Repository Update (Version 2)

This notebook demonstrates the workflow for ingesting updated content into the repository:

1. **Ingest Updated Content**: Add version 2 of edited documents to `content_repo`

**Note**: This notebook ONLY handles content repository ingestion. Chunking and checksum computation is handled in [4_change_detection.ipynb](4_change_detection.ipynb).

## Purpose

This notebook simulates a **content update cycle**:
- Version 1 files were ingested in `0_content_repo_ingestion.ipynb`
- Content has been edited (policy changes, updates, corrections)
- Version 2 files need to be ingested into `content_repo`
- Chunking and change detection will be done in the next notebook

## Prerequisites

- Database schema created (`content_repo`, `content_chunks` tables)
- Version 1 content already ingested (notebook 0)
- Version 2 markdown files exist on disk (edited versions)

## Key Concepts

- **File Versioning**: `raw_file_version_nbr` distinguishes v1 vs v2 of same file
- **Content Repository**: Stores metadata and references to source files
- **Separation of Concerns**: This notebook only ingests; chunking/checksums are separate

## Workflow

```
v1 Content (already done)     →     v2 Content (this notebook)
├─ content_repo (v1 files)    →     ├─ content_repo (v2 files)
                                            ↓
                                    Next: 4_change_detection.ipynb
                                            ↓
                                    Chunk + Compute checksums
                                            ↓
                                    Detect changes
                                            ↓
                                    Identify affected FAQs
```

In [ ]:
# Standard library imports
import sys
from pathlib import Path
import sqlite3

# Third-party imports
import pandas as pd

# Local imports - add parent to path
sys.path.append(str(Path.cwd().parent))

from granular_impact.data_ingestion import ContentRepoIngestion

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', 1000)

print("✓ Imports successful")

In [ ]:
# Configuration
PROJECT_ROOT = Path.cwd().parent if 'granular_impact' in str(Path.cwd()) else Path.cwd()
DB_PATH = PROJECT_ROOT / "databases" / "my_database.db"
DATA_DIR = PROJECT_ROOT / "granular_impact" / "data_ingestion" / "data"
MARKDOWN_DIR = DATA_DIR / "markdown"

print(f"Project Root: {PROJECT_ROOT}")
print(f"Database: {DB_PATH}")
print(f"Data Directory: {DATA_DIR}")
print(f"Markdown Directory: {MARKDOWN_DIR}")

# Verify paths exist
assert DB_PATH.exists(), f"Database not found: {DB_PATH}"
assert DATA_DIR.exists(), f"Data directory not found: {DATA_DIR}"
assert MARKDOWN_DIR.exists(), f"Markdown directory not found: {MARKDOWN_DIR}"

print("\n✓ All paths verified")

## Helper Functions

In [ ]:
def print_section(title: str):
    """Print a formatted section header."""
    print("\n" + "=" * 80)
    print(f"  {title}")
    print("=" * 80)


def print_result(result: dict, operation: str):
    """Print ingestion result in a formatted way."""
    if result['success']:
        print(f"\n✓ {operation} SUCCESS")
        print(f"  Rows inserted: {result['rows_inserted']}")
        print(f"  Message: {result['message']}")
    else:
        print(f"\n✗ {operation} FAILED")
        print(f"  Message: {result['message']}")
        raise Exception(f"{operation} failed: {result['message']}")


def print_stats(stats: dict, title: str):
    """Print statistics in a formatted table."""
    print(f"\n{title}:")
    for key, value in stats.items():
        print(f"  {key:30s}: {value}")


print("✓ Helper functions defined")

---
## Step 1: Ingest Version 2 Content Repository

Ingest the **edited/updated** version of content files into `content_repo`.

**Key points:**
- Version 2 files have `raw_file_version_nbr = 2`
- Same `raw_file_nme` but different version number
- Updated content will have different checksums in chunks
- Enables side-by-side comparison with v1

In [4]:
print_section("STEP 1: CONTENT REPOSITORY INGESTION")

# Load the sample content repo CSV
content_csv_path = DATA_DIR / "sample_content_repo_v2_edited.csv"
content_df = pd.read_csv(content_csv_path)

print(f"\nLoaded {len(content_df)} content records from CSV")
print("\nFirst few records:")
print(content_df[['raw_file_nme', 'title_nme', 'content_checksum']].head())


  STEP 1: CONTENT REPOSITORY INGESTION

Loaded 15 content records from CSV

First few records:
             raw_file_nme                                              title_nme  content_checksum
0   Employee_Handbook.pdf                       Employee Handbook 2025 - Updated               NaN
1  IT_Security_Policy.pdf    IT Security Policy - Enhanced Password Requirements               NaN
2  Remote_Work_Policy.pdf  Remote Work Policy - Updated Guidelines (3 days/week)               NaN
3       Travel_Policy.pdf    Business Travel Policy - Domestic and International               NaN
4      Parental_Leave.pdf   Parental Leave Policy - Enhanced Benefits (16 weeks)               NaN


### View Loaded Data

In [ ]:
# Display summary of v2 data
print("\nVersion 2 Content Summary:")
print(f"  Total files: {len(content_df)}")
print(f"  Unique filenames: {content_df['raw_file_nme'].nunique()}")
print(f"  Version numbers: {content_df['raw_file_version_nbr'].unique()}")

print("\nVersion 2 files:")
print(content_df[['raw_file_nme', 'raw_file_version_nbr', 'title_nme']].to_string(index=False))

### Ingest into Database

In [6]:
# Initialize ingestion
content_ingestion = ContentRepoIngestion(str(DB_PATH))

# Get stats before ingestion
stats_before = content_ingestion.get_stats()
print_stats(stats_before, "Content Repo - BEFORE Ingestion")

# Option: Clear existing content (uncomment if needed)
# print("\nClearing existing content_repo data...")
# clear_result = content_ingestion.clear()
# print(f"  {clear_result['message']}")

# Ingest from DataFrame
print("\nIngesting content_repo data...")
result = content_ingestion.ingest_from_dataframe(content_df)
print_result(result, "Content Repo Ingestion")

# Get stats after ingestion
stats_after = content_ingestion.get_stats()
print_stats(stats_after, "Content Repo - AFTER Ingestion")


Content Repo - BEFORE Ingestion:
  total_records                 : 14
  unique_files                  : 14
  file_types                    : ['pdf']
  statuses                      : ['Active']

Ingesting content_repo data...

✓ Content Repo Ingestion SUCCESS
  Rows inserted: 15
  Message: Successfully inserted 15 rows into content_repo

Content Repo - AFTER Ingestion:
  total_records                 : 29
  unique_files                  : 16
  file_types                    : ['pdf']
  statuses                      : ['Active']


In [7]:
import sqlite3  # Import here to ensure it's available

print(f"\n{'='*80}")
print("  VERIFICATION QUERIES")
print(f"{'='*80}\n")

conn = sqlite3.connect(str(DB_PATH))

# Query 1: Sample checksums
print("Query 1: Sample checksums from database\n")
query = """
SELECT *
FROM content_repo
"""
result_df = pd.read_sql_query(query, conn)
print(result_df.to_string(index=False))


  VERIFICATION QUERIES

Query 1: Sample checksums from database

 ud_source_file_id              raw_file_nme raw_file_type  raw_file_version_nbr                                                     raw_file_path                                             extracted_markdown_file_path                                             title_nme content_checksum file_status          created_dt    last_modified_dt
                 1     Employee_Handbook.pdf           pdf                     1     granular_impact/data_ingestion/data/raw/Employee_Handbook.pdf     granular_impact/data_ingestion/data/markdown/Employee_Handbook_v1.md                                Employee Handbook 2024             None      Active 2025-10-27 09:44:40 2025-10-27 09:44:40
                 2    IT_Security_Policy.pdf           pdf                     1    granular_impact/data_ingestion/data/raw/IT_Security_Policy.pdf    granular_impact/data_ingestion/data/markdown/IT_Security_Policy_v1.md            IT Security Polic